In [1]:
using LinearAlgebra, SparseArrays

In [2]:
using LowLevelFEM
gmsh.initialize()

gmsh.open("tarto.geo")

Info    : Reading 'tarto.geo'...
Info    : Meshing 1D...- Splitting solids                                                                                             
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (TrimmedCurve)
Info    : [ 10%] Meshing curve 6 (TrimmedCurve)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (TrimmedCurve)
Info    : [ 10%] Meshing curve 10 (TrimmedCurve)
Info    : [ 20%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Circle)
Info    : [ 20%] Meshing curve 13 (Circle)
Info    : [ 20%] Meshing curve 14 (Circle)
Info    : [ 20%] Meshing curve 15 (Circle)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Circle)
Info    : [ 20%] Meshing curve 18 (Line)
Info    : [ 20%] Meshing curve 19 (Circle)
Info    : [ 20%] M

In [3]:
mat = Material("part")
prob = Problem([mat])

Problem("tarto", :Solid, 3, 3, Material[Material("part", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 27263, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [4]:
supp1 = displacementConstraint("base", uz=0)
supp2 = displacementConstraint("hole1", ux=0, uy=0)
supp3 = displacementConstraint("perim", uz=-0.00001)

load1 = load("hole2", fy=-5);

In [5]:
F = loadVector(prob, [load1])
showDoFResults(F, name="load");

In [6]:
resultant(F, "hole2")

3-element Vector{Float64}:
    0.0
 -942.413625487596
    0.0

In [7]:
λ = mat.λ
μ = mat.μ;

In [8]:
@time K = poissonMatrix(prob, coefficient=2μ) + gradDivMatrix(prob, coefficient=λ) - curlCurlMatrix(prob, coefficient=μ)

 18.453312 seconds (46.40 M allocations: 5.419 GiB, 15.78% gc time, 3.99% compilation time)


sparse([1, 2, 3, 169, 170, 171, 223, 224, 225, 280  …  76602, 76603, 76604, 76605, 81709, 81710, 81711, 81787, 81788, 81789], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789], [270543.11346134474, -11159.804377662445, 102644.77760057495, -4597.344962031356, 264.3247288412763, -4646.554931874546, 5175.806547185506, -5921.919862186506, 11334.867428781494, 20476.172811385757  …  -298334.2519184167, -186029.82034847172, -224222.7973486119, -280188.98738505895, -59856.25414105746, 77623.63039804924, 112989.45363675174, -107045.1003009175, 270525.9942690422, 1.5092618746204376e6], 81789, 81789)

In [9]:
@time K0 = stiffnessMatrix(prob)

  6.337766 seconds (17.46 M allocations: 8.493 GiB, 25.39% gc time)


sparse([1, 2, 3, 169, 170, 171, 223, 224, 225, 280  …  76602, 76603, 76604, 76605, 81709, 81710, 81711, 81787, 81788, 81789], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789, 81789], [270543.11346134485, -11159.80437766245, 102644.777600575, -4597.344962031357, 264.3247288412764, -4646.554931874547, 5175.806547185517, -5921.919862186509, 11334.867428781497, 20476.17281138576  …  -298334.2519184168, -186029.82034847175, -224222.79734861193, -280188.98738505895, -59856.25414105749, 77623.63039804928, 112989.45363675182, -107045.10030091746, 270525.9942690423, 1.5092618746204376e6], 81789, 81789)

In [10]:
norm(K[:, :] - K0[:, :])

1.127814762713756e-7

In [11]:
K_theory = poissonMatrix(prob, coefficient=μ) +
           gradDivMatrix(prob, coefficient=λ + μ) -
           curlCurlMatrix(prob, coefficient=μ)

norm(K_theory[:, :] - K0[:, :])

1.6603483160094115e8

In [12]:
f = loadVector(prob, [load1]);

In [13]:
all = constrainedDoFs(prob, [BoundaryCondition("part", ux=0, uy=0, uz=0)])
bc = constrainedDoFs(prob, [supp1, supp2, supp3])
free = setdiff(all, bc);

In [14]:
u = applyBoundaryConditions(prob, [supp1, supp2, supp3]);

In [15]:
DoFs(u)[free] = K[free, free] \ DoFs(f)[free];

In [16]:
showDoFResults(u, name="displacement")

2

In [17]:
@time A = (u ∘ ∇ + ∇ ∘ u) / 2

 10.443396 seconds (22.21 M allocations: 1.660 GiB, 2.84% gc time, 45.13% compilation time)


elementwise TensorField
[[3.383102068800009e-7; 5.41047498428069e-7; … ; -1.1957431261576086e-5; 3.5564953638085356e-5;;], [2.9870183876356426e-6; 1.8592654330909687e-5; … ; -1.9107435703077165e-5; 4.266973601571699e-5;;], [1.3064995123075518e-5; -2.881997745767743e-5; … ; -2.5032479062784153e-5; -0.00019841780115497504;;], [1.7127378517077654e-5; 4.587382462651014e-6; … ; -1.1333005651458464e-6; 6.447111163466161e-7;;], [-6.8253343635271915e-6; 2.3328215027220822e-7; … ; -1.9528692799472406e-5; 0.00014562220978927838;;], [-6.18668469353618e-6; 5.102796813598843e-7; … ; -1.9188326838793124e-5; 0.00015913039930481225;;], [1.116328783002977e-5; 3.3299668274826084e-6; … ; 2.7831576005368712e-5; -6.44554547830742e-5;;], [-2.281490070552684e-5; -2.2186519798303244e-6; … ; 3.2702699751475717e-5; 4.074071048673541e-5;;], [-4.3680174468822095e-5; -7.878362604895564e-8; … ; -1.400539996125865e-5; 8.596692063296291e-5;;], [5.3492529869805675e-5; -4.009021688246815e-7; … ; -1.8146955448530594e-5;

In [18]:
@time solveStrain(u)

  5.795953 seconds (15.68 M allocations: 1.073 GiB, 9.83% gc time, 61.17% compilation time)


elementwise TensorField
[[3.383102068800001e-7; 5.410474984250679e-7; … ; -1.1957431261581074e-5; 3.556495363808525e-5;;], [2.987018387635629e-6; 1.85926543309097e-5; … ; -1.91074357030772e-5; 4.266973601571706e-5;;], [1.3064995123075545e-5; -2.881997745767667e-5; … ; -2.503247906278502e-5; -0.0001984178011549733;;], [1.7127378517077647e-5; 4.587382462651002e-6; … ; -1.133300565145849e-6; 6.447111163466123e-7;;], [-6.8253343635271915e-6; 2.3328215026895138e-7; … ; -1.952869279947095e-5; 0.00014562220978927816;;], [-6.18668469353618e-6; 5.102796813601096e-7; … ; -1.9188326838792853e-5; 0.0001591303993048116;;], [1.1163287830029777e-5; 3.329966827482594e-6; … ; 2.783157600536875e-5; -6.445545478307415e-5;;], [-2.2814900705526858e-5; -2.218651979830364e-6; … ; 3.270269975147576e-5; 4.074071048673541e-5;;], [-4.368017446882208e-5; -7.878362606739131e-8; … ; -1.4005399961261253e-5; 8.596692063296205e-5;;], [5.34925298698057e-5; -4.0090216882533204e-7; … ; -1.814695544853222e-5; -0.000161319

In [19]:
showElementResults(A, name="strain")

3

In [20]:
I = TensorField(prob, "part", [1 0 0; 0 1 0; 0 0 1])
S = 2μ * A + λ * trace(A) * I;

In [21]:
showDoFResults(S, name="stress")

4

In [22]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.15.0-git
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib tinyobjloader
FLTK version  : 1.3.8
OCC version   : 7.9.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
Fontconfig warning: using without calling FcInit()


In [23]:
gmsh.finalize()